In [86]:
import base64
import datetime
import json
import os
import random
import time

import scipy
from scipy import ndimage
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from pylab import *
from matplotlib.dates import date2num , DateFormatter
from PIL import Image

np.random.seed(1337)

%matplotlib inline
sns.set(font_scale=1.0)
rcParams['figure.figsize'] = 8, 6
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

pd.options.display.max_colwidth = 1000

In [87]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix

from sklearn import preprocessing

# Load the Data

In [88]:
d = pd.concat([pd.read_csv('tmp/train_ids.csv', index_col='id'),
               pd.read_csv('tmp/test_ids.csv', index_col='id')])

In [89]:
e = pd.read_csv('tmp/features_bathrooms.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [90]:
e = pd.read_csv('tmp/features_bedrooms.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [91]:
e = pd.read_csv('tmp/features_building_id.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [92]:
e = pd.read_csv('tmp/features_created.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [93]:
e = pd.read_csv('tmp/features_description.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [94]:
e = pd.read_csv('tmp/features_display_address.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [95]:
e = pd.read_csv('tmp/features_features.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [96]:
e = pd.read_csv('tmp/features_latlon.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [97]:
e = pd.read_csv('tmp/features_manager_id.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [98]:
e = pd.read_csv('tmp/features_photos.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [99]:
e = pd.read_csv('tmp/features_price.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [100]:
e = pd.read_csv('tmp/features_street_address.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [101]:
d.interest_level.value_counts()

low       34284
medium    11229
high       3839
Name: interest_level, dtype: int64

In [102]:
# d = pd.concat([d, tf_idfs], axis=1)
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124011 entries, 0 to 124010
Columns: 102 entries, listing_id to daddr_saddr_same
dtypes: float64(49), int64(42), object(11)
memory usage: 97.5+ MB


In [103]:
tr = d[d.interest_level.notnull()].copy()
print len(tr)

49352


In [104]:
trl = tr.interest_level.copy()
trf = tr
del trf['interest_level']

In [105]:
te = d[d.interest_level.isnull()].copy()
print len(te)

74659


In [106]:
tef = te
del tef['interest_level']

In [107]:
categorical = ["manager_id", "building_id"]
for f in categorical:
        if trf[f].dtype=='object':
            #print(f)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(trf[f].values) + list(tef[f].values))
            trf[f] = lbl.transform(list(trf[f].values))
            tef[f] = lbl.transform(list(tef[f].values))
            


In [108]:
trf.columns

Index([u'listing_id', u'bathrooms', u'num_bathrooms_bucket', u'bedrooms',
       u'num_bedrooms_bucket', u'building_id', u'num_apts_in_building',
       u'num_apts_in_building_q10', u'created', u'created_utc',
       ...
       u'price_q10_all', u'price_q100_all', u'price_q10', u'price_q100',
       u'street_address', u'saddr_num_words', u'saddr_has_number',
       u'saddr_has_ordinal', u'saddr_num_upper_words', u'daddr_saddr_same'],
      dtype='object', length=101)

In [109]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy import sparse
tfidf = CountVectorizer(stop_words='english', max_features=400)
tr_sparse = tfidf.fit_transform(trf["features"])
te_sparse = tfidf.transform(tef["features"])

In [56]:
for c in trf.columns:
    if trf.dtypes[c] not in ('int64', 'float64'):
        del trf[c]           
            
for c in tef.columns:
    if tef.dtypes[c] not in ('int64', 'float64'):
        del tef[c]

# Run Classification

In [57]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0, num_rounds=1000):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 4
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.8
    param['colsample_bytree'] = 0.8
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=50)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [58]:
train_X = sparse.hstack([trf,tr_sparse]).tocsr()
test_X = sparse.hstack([tef,te_sparse]).tocsr()

target_num_map = {'high':0, 'medium':1, 'low':2}
train_y = np.array(trl.apply(lambda x: target_num_map[x]))
train_y

array([1, 2, 1, ..., 1, 1, 0])

In [59]:
preds, model = runXGB(train_X, train_y, test_X, num_rounds=1000)

In [60]:
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = pd.read_csv('input/sample_submission.csv').listing_id.values
out_df.to_csv("output/xgb_starter_3_own_features_tfidf_manager_building.csv", index=False)

In [67]:
def cvXGB(train_X, train_y, test_X, seed_val=0, num_rounds=1000):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 4
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.8
    param['colsample_bytree'] = 0.8
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    eval_hist = xgb.cv(plst, xgtrain, num_rounds, nfold=3)

    
    return eval_hist


In [68]:
eval_history = cvXGB(train_X, train_y, test_X, num_rounds=1000)
eval_history

,test-mlogloss-mean,test-mlogloss-std,train-mlogloss-mean,train-mlogloss-std
0,1.037760,0.000793,1.037353,0.000418
1,0.986576,0.001264,0.985883,0.000778
2,0.942336,0.001204,0.941165,0.000693
3,0.904473,0.001217,0.902864,0.000709
4,0.871742,0.001138,0.869791,0.001129
5,0.843040,0.001168,0.840671,0.001124
6,0.817794,0.001225,0.815117,0.001172
7,0.795887,0.001356,0.792719,0.001100
8,0.776449,0.001406,0.772991,0.001156
9,0.759514,0.001495,0.755585,0.001076
